# 04 - AI Orchestration with Azure AI Search
**(Langchain / Python version)**

In this lab, we will do a deeper dive into using Azure AI Search as a vector store, the different search methods it supports and how you can use it as part of the Retrieval Augmented Generation (RAG) pattern for working with large language models.

## Create an Azure AI Search Vector Store in Azure

First, we will create an Azure AI Search service in Azure. The following are command line instructions and require the Azure CLI to be installed.

**NOTE:** Before running the commands, replace the **`<INITIALS>`** with your own initials or some random characters, as we need to provide a unique name for the Azure AI Search service.

RESOURCE_GROUP="rg-yimi-ibit-hackathon"
LOCATION="westeurope"
NAME="aisearch-yimi-vectorstore"
!az search service create -g $RESOURCE_GROUP -n $NAME -l $LOCATION --sku Basic --partition-count 1 --replica-count 1

Next, we need to find and update the following values in the `.env` file with the Azure AI Search **name**, **endpoint** and **admin key** values, which you can get from the Azure portal. You also need to provide an **index name** value. The index will be created during this lab, so you can use any name you like.

```
AZURE_AI_SEARCH_SERVICE_NAME = "<YOUR AZURE AI SEARCH SERVICE NAME - e.g. ai-vectorstore-xyz>"
AZURE_AI_SEARCH_ENDPOINT = "<YOUR AZURE AI SEARCH ENDPOINT URL - e.g. https://ai-vectorstore-xyz.search.windows.net"
AZURE_AI_SEARCH_INDEX_NAME = "<YOUR AZURE AI SEARCH INDEX NAME - e.g. ai-search-index>"
AZURE_AI_SEARCH_API_KEY = "<YOUR AZURE AI SEARCH ADMIN API KEY - e.g. get this value from the Azure portal>"
```

## Load environment variable values
As with previous labs, we'll use the values from the `.env` file in the root of this repository.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
# Load environment variables
if load_dotenv(dotenv_path="../../../.env"):
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")


azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")


This lab exercise will use the following values:
Azure OpenAI Endpoint: https://aoai-ibit-hackathon.openai.azure.com/
Azure AI Search: aisearch-yimi-vectorstore


First, we will load the data from the movies.csv file and then extract a subset to load into the Azure AI Search index. We do this to help avoid the Azure OpenAI embedding limits and long loading times when inserting data into the index. We use a Langchain document loader to do this.

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path='./movies.csv',
    source_column='original_title',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'fieldnames': [
            'id', 'original_language', 'original_title', 'popularity',
            'release_date', 'genre',
            'overview', 'revenue', 'runtime', 'tagline'
        ]
    }
)
data = loader.load()

# Rather than load all 500 movies into Azure AI search, we will use a
# smaller subset of movie data to make things quicker. The more movies you load,
# the more time it will take for embeddings to be generated.

data = data[1:51]
print('Loaded %s movies.' % len(data))

Loaded 50 movies.


During this lab, we will need to work with embeddings. We use embeddings to create a vector representation of a piece of text. We will need to create embeddings for the documents we want to store in our Azure AI Search index and also for the queries we want to use to search the index. We will create an Azure OpenAI client to do this.

In [3]:
from langchain_openai import AzureOpenAIEmbeddings

AZURE_OPENAI_EMBEDDING_MODEL = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING_MODEL_VERSION = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_VERSION")

azure_openai_embeddings = AzureOpenAIEmbeddings(    
    azure_deployment = AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME,
    openai_api_version = AZURE_OPENAI_EMBEDDING_MODEL_VERSION,
    model= AZURE_OPENAI_EMBEDDING_MODEL,
    chunk_size=1000,
)

## Create an Azure AI Search index => UI TODO

Next, we'll step through the process of configuring an Azure AI Search index to store our movie data and then loading the data into the index. 

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SemanticPrioritizedFields,
    SemanticSearch,
    SemanticField,
    SemanticConfiguration,
    SimpleField,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex
)
from azure.search.documents.models import (
    VectorizedQuery
)

When configuring an Azure AI Search index, we need to specify the fields we want to store in the index and the data types for each field. These match the fields in the movie data, containing values such as the movie title, genre, year of release and so on.

To use Azure AI Search as a vector store, we will also need to define a field to hold the vector representaion of the movie data. We indicate to Azure AI Search that this field will contain vector data by providing details of the vector dimensions and a profile. We'll also define the vector search configuration and profile with default values.

**NOTE:** It is possible just to use Azure AI Search as a vector store only, in which case we probably wouldn't need to define all of the index fields below. However, in this lab, we're also going to demonstrate Hybrid Search, a feature which makes use of both traditional keyword based search in combination with vector search.

In [ ]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="overview", type=SearchFieldDataType.String),
    SearchableField(name="genre", type=SearchFieldDataType.String),
    SearchableField(name="tagline", type=SearchFieldDataType.String),
    SearchableField(name="release_date", type=SearchFieldDataType.DateTimeOffset, sortable=True),
    SearchableField(name="popularity", type=SearchFieldDataType.Double, sortable=True),
    SearchableField(name="vote_average", type=SearchFieldDataType.Double, sortable=True),
    SearchableField(name="vote_count", type=SearchFieldDataType.Int32, sortable=True),
    SearchableField(name="runtime", type=SearchFieldDataType.Int32, sortable=True),
    SearchableField(name="revenue", type=SearchFieldDataType.Int64, sortable=True),
    SearchableField(name="original_language", type=SearchFieldDataType.String),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_profile_name="movies-vector-profile"),
]

vector_search = VectorSearch(
    profiles=[VectorSearchProfile(name="movies-vector-profile", algorithm_configuration_name="movies-vector-config")],
    algorithms=[HnswAlgorithmConfiguration(name="movies-vector-config")],
)

We're going to be using Semantic Ranking, a feature of Azure AI Search that improves search results by using language understanding to rerank the search results. We provide a Semantic Search Configuration to help the ranking model understand the movie data, by telling it which fields contain the movie title, which fields contain keywords and which fields contain general free text content.

In [ ]:
semantic_config = SemanticConfiguration(
    name="movies-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="genre")],
        content_fields=[SemanticField(field_name="title"),
                        SemanticField(field_name="overview"),
                        SemanticField(field_name="tagline"),
                        SemanticField(field_name="genre"),
                        SemanticField(field_name="release_date"),
                        SemanticField(field_name="popularity"),
                        SemanticField(field_name="runtime"),
                        SemanticField(field_name="revenue"),
                        SemanticField(field_name="original_language")],
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

Finally, we'll go ahead and create the index by creating an instance of the `SearchIndex` class and adding the keyword and vectors fields and the semantic search profile.

In [ ]:
# Create the search index with the desired vector search and semantic configurations
index = SearchIndex(
    name=azure_ai_search_index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

index_client = SearchIndexClient(
    azure_ai_search_endpoint,
    AzureKeyCredential(azure_ai_search_api_key)
)

result = index_client.create_or_update_index(index)

print(f'Index {result.name} created.')

The index is now ready, so next we need to prepare the movie data to load into the index.

**NOTE**: During this phase, we send the data for each movie to an Azure OpenAI embeddings model to create the vector data. This may take some time due to rate limiting in the API.

## COSMOS DB

In [5]:
def generate_embedding(content):
    """
    Generates an embedding for the given content using Azure OpenAI.
    """
    try:
        return azure_openai_embeddings.embed_query(content)  # Ensure this returns a list of floats
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return []  # Return an empty list if embedding fails

In [6]:
# Loop through all of the movies and create a new item for each one.

def parse_movie(movie, vector):
    """
    Parses a movie object into a structured dictionary with proper data types.
    """
    try:
        content = movie.page_content
        fields = dict(line.split(": ", 1) for line in content.split("\n") if ": " in line)

        # Convert data types correctly
        movie_data = {
            "id": int(float(fields.get("id", "0"))),  # Ensure it's an integer
            "original_language": fields.get("original_language", "").strip(),
            "original_title": fields.get("original_title", "").strip(),
            "popularity": fields.get("popularity", "0"),
            "release_date": fields.get("release_date", "").strip(),
            "vote_average": fields.get("vote_average", "0"),
            "vote_count": fields.get("vote_count", "0"),
            "genre": fields.get("genre", "[]"),  # Convert to a list
            "overview": fields.get("overview", "").strip(),
            "revenue": fields.get("revenue", "0"),
            "runtime": fields.get("runtime", "0"),
            "tagline": fields.get("tagline", "").strip(),
            "vector": vector
        }
        
        return movie_data

    except Exception as e:
        print(f"Error parsing movie: {e}")
        return {}

# Parse and vectorize all movies
parsed_movies = []
for movie in data:
    content = movie.page_content
    vector = generate_embedding(content)  # Generate embedding before parsing
    parsed_movies.append(parse_movie(movie, vector))  # Pass vector to parse_movie

print(parsed_movies[0])  # Print first parsed movie to verify

print(f"New items structure with embeddings created for {len(parsed_movies)} movies.")

{'id': 381284, 'original_language': 'en', 'original_title': 'Hidden Figures', 'popularity': '49.802', 'release_date': '2016-12-10', 'vote_average': '0', 'vote_count': '0', 'genre': '8.1', 'overview': '7310.0', 'revenue': "['Drama', 'History']", 'runtime': 'The untold story of Katherine G. Johnson, Dorothy Vaughan and Mary Jackson – brilliant African-American women working at NASA and serving as the brains behind one of the greatest operations in history – the launch of astronaut John Glenn into orbit. The visionary trio crossed all gender and race lines to inspire generations to dream big.', 'tagline': '230698791.0', 'vector': [-0.03348075598478317, 0.017375215888023376, -0.00873591098934412, 0.008784214034676552, -0.003141408786177635, -0.018851902335882187, 0.007638746406883001, 0.05183583125472069, -0.010875036008656025, 0.03541287034749985, 0.013034862466156483, -0.01817566342651844, 0.010171194560825825, 0.015498306602239609, -0.043389737606048584, 0.016077939420938492, -0.0030120

In [7]:
from azure.cosmos import CosmosClient, PartitionKey

# Replace with your actual Cosmos DB values
COSMOS_DB_ENDPOINT = os.getenv("COSMOS_DB_URL")
COSMOS_DB_KEY = os.getenv("COSMOS_DB_KEY")
DATABASE_NAME = os.getenv("COSMOS_DB_NAME")
CONTAINER_NAME = os.getenv("COSMOS_DB_CONTAINER")
# Create client
client = CosmosClient(COSMOS_DB_ENDPOINT, COSMOS_DB_KEY)
# Create (or get) a database
database = client.create_database_if_not_exists(id=DATABASE_NAME)
# Create (or get) a container with a partition key
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id")  # if you want to partition on 'id'
    # You can change /id to /genre or some other column if desired.
)

# Upload Data to CosmosDB
for item in parsed_movies:
    # Ensure 'id' is always a string (CosmosDB requires string IDs)
    item["id"] = str(item["id"])

    try:
        container.upsert_item(item)  # Insert or update if exists
        print(f"Uploaded: {item['original_title']}")
    except Exception as e:
        print(f"Error uploading {item['original_title']}: {e}")

print(f"Successfully uploaded {len(parsed_movies)} movies into CosmosDB! 🚀")

Uploaded: Hidden Figures
Uploaded: Gridlocked
Uploaded: Joker
Uploaded: The Sand
Uploaded: America: The Motion Picture
Uploaded: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Uploaded: Under Siege 2: Dark Territory
Uploaded: The Enforcer
Uploaded: Ruby Sparks
Uploaded: They Came Together
Uploaded: The Handmaid's Tale
Uploaded: Броненосец Потёмкин
Uploaded: Enemy of the State
Uploaded: Pistol Whipped
Uploaded: 맛있는 비행
Uploaded: Wheelman
Uploaded: Raising Arizona
Uploaded: Rampage
Uploaded: Evolution
Uploaded: Man on Wire
Uploaded: Work It
Uploaded: Step Sisters
Uploaded: Pirates of the Caribbean: Tales of the Code – Wedlocked
Uploaded: I Don't Know How She Does It
Uploaded: The Wrestler
Uploaded: The Sisterhood of the Traveling Pants
Uploaded: Charlie Wilson's War
Uploaded: El Infierno
Uploaded: Una última y nos vamos
Uploaded: Immortals
Uploaded: Pandorum
Uploaded: Women
Uploaded: Hotel Transylvania 3: Summer Vacation
Uploaded: Alex Strangelove
Uploaded: The Painted Veil
Uploaded: Johnson Family Vacati

We can write out the contents of one of the documents to see what it looks like. You can see that it contains the movie data at the top and then a long array containing the vector data.

In [ ]:
print(parsed_movies[0])

Now we have the movie data stored in the correct format, so let's load it into the Azure AI Search index we created earlier.

In [9]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

search_client = SearchClient(
    azure_ai_search_endpoint,
    azure_ai_search_index_name,
    AzureKeyCredential(azure_ai_search_api_key)
)


# result = search_client.upload_documents(parsed_movies)

print(f"Successfully loaded {len(data)} movies into Azure AI Search index.")

Successfully loaded 50 movies into Azure AI Search index.


## Vector store searching using Azure AI Search

We've loaded the movies into Azure AI Search, so now let's experiment with some of the different types of searches you can perform.

First we'll just perform a simple keyword search.

In [10]:
query = "hero"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Movie: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 8.0
----------


We get some results, but they're not necessarily movies about heroes. It could be that there is some text in the index for these results that relates to the word "hero". For example, the description might mention "heroic deeds" or something similar.

Let's now try the same again, but this time we'll ask a question instead of just searching for a keyword.

In [11]:
query = "What are the best movies about superheroes?"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Movie: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

Movie: Pandorum
Genre: 6.5
----------
Movie: The Bucket List
Genre: 7.2
----------
Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 8.0
----------
Movie: Evolution
Genre: 6.0
----------
Movie: Catch Me If You Can
Genre: 7.9
----------


As before, you will likely get mixed results. Some of the movies returned could be about heroes, but others may not be. This is because the search is still based on keywords.

Next, let's try a vector search.

In [13]:
from azure.search.documents.models import VectorizedQuery

query = "What are the best movies about superheroes?"

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

results = list(search_client.search(search_text=None, select=["vector"], top=1))
print(results)


# Note the `None` value for the `search_text` parameter. This is because we're not sending the query text to Azure AI Search. We're sending the embedded version of the query text instead via the `vector_queries` parameter.

results = list(search_client.search(
    search_text=None,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["original_title", "genre"],
    top=5
))


print(results)

for result in results:
    print("Movie: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

[{'vector': [-0.0035162903, 0.008074959, -0.0018606457, 0.007414786, 0.025725903, -0.016691955, -0.0030976017, 0.03897106, -0.03485714, 0.052396894, 0.044863973, -0.041834127, -0.024516743, -0.008019365, -0.03366188, -0.0017859419, 0.010194462, 0.019680107, 0.0028856513, -0.038415123, 0.03218865, 0.004162565, -0.015246524, -0.01519093, 0.014273637, 5.8959537e-05, -0.004628161, 0.034301203, -0.0091381855, -0.02412759, -0.008102756, -0.016427886, -0.02170927, -0.01990248, 0.04136158, 0.026587602, 0.01751196, -0.008081908, 0.021598084, 0.0018901798, 0.022445885, 0.016316699, -0.023029616, 0.02266826, 0.019193664, -0.0035249768, 0.048449755, -0.030604234, -0.009159033, 0.027226929, 0.034356795, -0.05673319, -0.016539073, -0.007115971, 0.00074660365, -0.022765547, 0.009020049, -0.021611981, 0.031104576, -0.042362265, -0.007261904, -0.022015035, 0.010958873, 0.0077900426, 0.017095009, -0.022265205, 0.022126222, 0.016163817, -0.007699703, 0.03535748, 0.01699772, 0.027101843, 0.030659828, -0.0

It's likely that the raw vector search didn't return exactly what you were expecting. You were probably expecting a list of superhero movies, but now we're getting a list of movies that are **similar** to the vector we provided. Some of these may be hero movies, but others may not be. The vector search is returning the nearest neighbours to the vector we provided, so it's possible that at least one of the results is a superhero movie, and the others are similar to that movie in some way.

So, both the keyword search and the vector search have their limitations. The keyword search is limited to the keywords in the index, so it's possible that we might miss some movies that are about heroes. The vector search is limited to returning the nearest neighbours to the vector we provide, so it's possible that we might get some movies that are not about heroes.

## Hybrid search using Azure AI Search

To overcome the limitations of both keyword search and vector search, we can use a combination of both. This is known as Hybrid Search. Let's run the same query again, but this time we'll use Hybrid Search.

The only significant difference is that this time we will submit both the original query text and the embedding vector to Azure AI Search. Azure AI Search will then use both the query text and the vector to perform the search and combine the results.

In [15]:
query = "What are the best movies about superheroes?"

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

results = list(search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["original_title", "genre"],
    top=5
))

for result in results:
    print("Movie: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("Score: {}".format(result["@search.score"]))
    print("Reranked score: {}".format(result["@search.reranker_score"]))
    print("----------")

Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 8.0
Score: 0.032258063554763794
Reranked score: 2.213501214981079
----------
Movie: Evolution
Genre: 6.0
Score: 0.01587301678955555
Reranked score: 2.1586248874664307
----------
Movie: Immortals
Genre: 5.9
Score: 0.02753623202443123
Reranked score: 2.1576058864593506
----------
Movie: Joker
Genre: 8.2
Score: 0.025676939636468887
Reranked score: 1.9723514318466187
----------
Movie: The Bucket List
Genre: 7.2
Score: 0.016393441706895828
Reranked score: 1.9062429666519165
----------


Hopefully, you'll now see a much better set of results. Performing a hybrid search has allowed us to combine the benefits of both keyword search and vector search. But also, Azure AI Search performs a further step when using hybrid search. It makes use of a Semantic Ranker to further improve the search results. The Semantic Ranker uses a language understanding model to understand the query text and the documents in the index and then uses this information to rerank the search results. So, after performing the keyword and vector search, Azure AI Search will then use the Semantic Ranker to re-order the search results based on the context of the original query.

In the results above, you can see a `Reranked Score`. This is the score that has been calculated by the Semantic Ranker. The `Score` is the score calculated by the keyword and vector search. You'll note that the results are returned in the order determined by the reranked score.

## Bringing it All Together with Retrieval Augmented Generation (RAG) + Langchain (LC)

Now that we have our Vector Store setup and data loaded, we are now ready to implement the RAG pattern using AI Orchestration. At a high-level, the following steps are required:
1. Ask the question
2. Create Prompt Template with inputs
3. Get Embedding representation of inputted question
4. Use embedded version of the question to search Azure AI Search (ie. The Vector Store)
5. Inject the results of the search into the Prompt Template & Execute the Prompt to get the completion

In [17]:
# Implement RAG using Langchain (LC)

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain


azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

azure_openai = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

# Ask the question
query = "What are the best movies about superheroes?"

# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    Question: {original_question}

    Do not use any other data.
    Only use the movie data below when responding.
    Provide detailed information about the synopsis of the movie.
    {search_results}
    """,
)

# Search Vector Store
search_client = SearchClient(
    azure_ai_search_endpoint,
    azure_ai_search_index_name,
    AzureKeyCredential(azure_ai_search_api_key)
)

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

results = list(search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    include_total_count=True,
    vector_queries=[vector],
    select=["original_title","genre","overview","tagline","release_date","popularity","runtime","revenue","original_language"],
    top=5
))

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# Build the Prompt and Execute against the Azure OpenAI to get the completion
chain = prompt | azure_openai | output_parser
response = chain.invoke(input={"original_question": query, "search_results": results})
print (response)


Based on the provided movie data, here are the best superhero movies along with detailed synopses:

1. **僕のヒーローアカデミア THE MOVIE ～2人の英雄～ (My Hero Academia: Two Heroes)**
   - **Synopsis**: All Might and Deku, two of the central heroes in the *My Hero Academia* universe, accept an invitation to visit "I-Island," a floating city known for its cutting-edge technology and research into quirks (superpowers) and hero supplemental tools. The trip is centered around "I-Expo," a prestigious expo showcasing unique innovations. Despite the advanced security measures surrounding the island, a villain manages to infiltrate the system, turning a seemingly peaceful event into a dire situation. The responsibility of stopping the threat falls on the students of Class 1-A, who must act as heroes in dangerous, real-world conditions. The movie showcases themes of teamwork, heroism, and overcoming incredible odds.

This movie stands out as an animated superhero adventure that combines action, youthful energy

## Next Section

📣 [Deploy AI](../../04-deploy-ai/README.md)